In [3]:
import numpy as np
from zne_entanglement_iqm import run_on_iqm_hardware

def add_measure_in_basis(prep, basis: str):
    qc = prep.copy()
    n = qc.num_qubits
    if basis == "X":
        qc.h(range(n))
    elif basis == "Y":
        qc.sdg(range(n))
        qc.h(range(n))
    else:
        raise ValueError("basis must be 'X' or 'Y'")
    qc.measure_all()
    return qc

def bitstring_to_pm1(bitstr: str):
    # 0 -> +1 eigenvalue, 1 -> -1 eigenvalue
    return np.array([+1 if b == "0" else -1 for b in bitstr], dtype=int)

def estimate_J2_from_counts(counts: dict, N: int) -> float:
    """
    Estimates <J^2> for J = 1/2 sum_i sigma_i (in the measured basis).
    Uses: <J^2> = N/4 + (1/4) * sum_{i!=j} <s_i s_j>,
    where s_i ∈ {+1,-1} are measurement eigenvalues in that basis.
    """
    shots = sum(counts.values())
    accum = 0.0
    for bitstr, c in counts.items():
        v = bitstring_to_pm1(bitstr)
        s = int(v.sum())
        # sum_{i!=j} v_i v_j = (sum v)^2 - sum v_i^2 = s^2 - N
        accum += c * (s*s - N)
    avg_offdiag = accum / shots
    return float((N/4) + (1/4)*avg_offdiag)

def sep_bound_Jx2_plus_Jy2(N: int) -> float:
    # N(N+1)/4
    return N * (N + 1) / 4

def run_oracle_witness_Jx2Jy2_on_iqm(
    prep,
    *,
    server_url: str,
    quantum_computer: str,
    token: str,
    shots: int = 4000,
    optimization_level: int = 1,
    seed_transpiler: int = 1,
    backend_name=None,
    cco = None
):
    N = prep.num_qubits

    cx = add_measure_in_basis(prep, "X")
    cy = add_measure_in_basis(prep, "Y")

    backend, result = run_on_iqm_hardware(
        [cx, cy],
        server_url=server_url,
        quantum_computer=quantum_computer,
        token=token,
        backend_name=backend_name,
        shots=shots,
        optimization_level=optimization_level,
        seed_transpiler=seed_transpiler,
        cco = cco
    )

    counts_x = result.get_counts(0)
    counts_y = result.get_counts(1)

    Jx2 = estimate_J2_from_counts(counts_x, N)
    Jy2 = estimate_J2_from_counts(counts_y, N)

    S = Jx2 + Jy2
    B = sep_bound_Jx2_plus_Jy2(N)

    Wexp = B - S  # <W> = bound - measured_sum
    entangled = (Wexp < 0)

    return {
        "backend": backend,
        "counts": {"X": counts_x, "Y": counts_y},
        "J2": {"X": Jx2, "Y": Jy2},
        "S": S,
        "bound": B,
        "W_exp": Wexp,
        "entangled": entangled,
    }


In [4]:
from dicke import dicke_state
from iqm.iqm_client import CircuitCompilationOptions
from iqm.iqm_client.models import DDMode, STANDARD_DD_STRATEGY, HeraldingMode

N = 3
k = 1
prep = dicke_state(N, k)


cco = CircuitCompilationOptions(
    dd_mode=DDMode.ENABLED,
    dd_strategy=STANDARD_DD_STRATEGY,
    heralding_mode=HeraldingMode.ZEROS,
)
out = run_oracle_witness_Jx2Jy2_on_iqm(
    prep,
    server_url="https://resonance.meetiqm.com",
    quantum_computer="garnet",
    token="rkSL242xyAzLHqJQrxHvHs2nREy4AbLE2aQcsA3unuoBnBZTQNV6spN3ItLtE2jJ",
    shots=10000,
    cco = cco
)

print("S = <Jx^2> + <Jy^2>:", out["S"])
print("Separable bound:", out["bound"])
print("<W> = bound - S:", out["W_exp"])
print("Entangled?", out["entangled"])


Progress in queue:   0%|          | 0/1 [00:09<?, ?it/s]


S = <Jx^2> + <Jy^2>: 3.232854316359471
Separable bound: 3.0
<W> = bound - S: -0.23285431635947118
Entangled? True
